## Artifact Dependencies

### References

- [neovis.js (GitHub)](https://github.com/neo4j-contrib/neovis.js)
- [vis-network (GitHub)](https://github.com/visjs/vis-network)
- [vis network documentation](https://visjs.github.io/vis-network/docs/network)
- [Neo4j Graph Algorithms Jupyter Notebooks (GitHub)](https://github.com/neo4j-graph-analytics/graph-algorithms-notebooks)


In [1]:
import os
from neo4j import GraphDatabase
from IPython.core.display import Javascript
import json

In [2]:
# Please set the environment variable "NEO4J_INITIAL_PASSWORD" in your shell 
# before starting jupyter notebook to provide the password for the user "neo4j". 
# It is not recommended to hardcode the password into jupyter notebook for security reasons.

neo4jUri = "bolt://localhost:7687"
neo4jUser = "neo4j"
neo4jPassword = os.environ.get("NEO4J_INITIAL_PASSWORD")

# Create the database driver to validate the connection
with GraphDatabase.driver(uri=neo4jUri, auth=(neo4jUser, neo4jPassword)) as driver:
    driver.verify_connectivity()

In [3]:
def neo4j_server_configuration(password, uri="bolt://localhost:7687", user="neo4j"):
    return {
        "neo4j": {
            "serverUrl": uri,
            "serverUser": user,
            "serverPassword": password
        }
    }

In [4]:
def visualization_configuration():
    return {
        "visConfig": {
            "nodes": {
                "shape": "hexagon",
                "font": {
                    "strokeWidth": 30,
                    "strokeColor": "#F0F0FF"
                },
                "size": 50,
                "borderWidth": 2
            },
            "edges": {
                "arrows": {
                    "to": { "enabled": True }
                },
                "scaling": {
                    "max": 15
                }
            },
            "physics": {
                "hierarchicalRepulsion": {
                    "nodeDistance": 300, # 100
                    "centralGravity": 0.5, # 0.2
                    "springLength": 180, # 200
                    "springConstant": 0.06, # 0.05
                    "damping": 0.09, # 0.09
                    "avoidOverlap": 0.1 # 0
                },
                "solver": "hierarchicalRepulsion" # barnesHut
            },
            "layout": {
                "hierarchical": {
                    "enabled": True,
                    "sortMethod": "directed"
                }
            }
        }
    }

In [5]:
def graph_query_configuration():
    return {
        "initialCypher": "MATCH (s:Artifact)-[r:DEPENDS_ON]->(d:Artifact) RETURN s,r,d",
        "labels": {
            "Artifact": {
                "label": "fileName"
            }
        },
        "relationships": {
            "DEPENDS_ON": {
                "value": "weight",
                "label": False
            }
        }
    }

In [6]:
htmlElement = {"containerId": "graph-visualization"}
serverConfiguration = neo4j_server_configuration(uri=neo4jUri, user=neo4jUser,password=neo4jPassword)

# Assemble the neovis.js configuration by joining the different parts of it
graphVisualizationConfiguration = {**htmlElement, **visualization_configuration(), **serverConfiguration, **graph_query_configuration()}

# Create a javascript variable containing the whole configuration in JSON format
Javascript("""window.graphVisualizationConfiguration={};""".format(json.dumps(graphVisualizationConfiguration)))

<IPython.core.display.Javascript object>

## Hierarchical Artifact Dependencies

The following hierarchical graph shows artifact dependencies with the most used basis/shared artifact at the bottom and the artifact the builds upon all other dependencies on top. 

In [7]:
%%html
<style type="text/css">
    #graph-visualization {
        width: 650px;
        height: 850px;
        border: 1px solid lightgray;
    }
</style>
<div id="graph-visualization"></div>

In [8]:
# Use the JavaScript library [neovis.js](https://github.com/neo4j-contrib/neovis.js) to render the graph into the HTML above with the following javascript block.

In [9]:
%%javascript
// Use JavaScript library neovis.js to render the graph into the HTML above
//requirejs(['./../lib/neovis/neovis.js'], function(NeoVis){    
requirejs(['https://unpkg.com/neovis.js@2.0.2'], function(NeoVis){  
    const configuration = window.graphVisualizationConfiguration;
    configuration.labels.Artifact = {
        [NeoVis.NEOVIS_ADVANCED_CONFIG]: {
          function: {
            // Print all properties for the title (when nodes are clicked)
            title: NeoVis.objectToTitleHtml,
            // Use "fileName" as label. Remove leading slash, trailing ".jar" and version number.
            label: (node) => node.properties.fileName.replace('/', '').replace('.jar', '').replace(/-[\d\\.]+/, '')
          },
        }
    }
    
    const viz = new NeoVis.default(configuration);
    viz.render();
  }, function (err) {
      throw new Error("Failed to load NeoVis:" + err);
  }
);

<IPython.core.display.Javascript object>

In [10]:
import time
time.sleep(6)